In [45]:
import numpy as np
import pandas as pd

importance_matrix = pd.read_excel("../data/feature_importance.xlsx", sheet_name=1)
N = importance_matrix.shape[0]

Judgment_matrix_A = importance_matrix.drop("Unnamed: 0", axis=1).values
# Judgment_matrix_A

In [46]:
Judgment_matrix_A[0]


array([1.        , 0.11111111, 0.33333333, 3.        , 9.        ,
       1.        , 0.14285714, 5.        , 1.        , 7.        ,
       0.2       ])

In [47]:
weights_W = []

for i in range(Judgment_matrix_A.shape[0]):
    weights_W.append(np.power(np.prod(Judgment_matrix_A[i]), 1/N))

std_weights_W = weights_W/np.sum(weights_W)
std_weights_W

array([0.05830091, 0.29790342, 0.05793639, 0.01153255, 0.01215675,
       0.06676792, 0.18382327, 0.10375309, 0.05677294, 0.01975171,
       0.13130106])

In [48]:
A_W = []
for i in range(Judgment_matrix_A.shape[0]):
    A_W.append(np.sum(std_weights_W*Judgment_matrix_A[i]))

lambda_max = np.sum(A_W / (N * std_weights_W))
lambda_max

13.360775872823936

In [49]:
CI = (lambda_max - N) / (N - 1)
RI = [0, 0, 0.68, 0.90, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49, 1.51, 1.54, 1.56]
CR = CI/RI[10]
CR

0.15634277303469776

In [50]:
arr1 = np.array([10, 20, 30, 40])
arr2 = np.array([2, 4, 5, 10])

arr1/arr2
RI[10]

1.51